# Imports

In [290]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [291]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path

from scipy.signal import savgol_filter

In [551]:
import matplotlib.patches as patches

In [292]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/1.2/data_fct_Adrien_point_by_point

matplotlib.rcParams.update({'font.size': 21})

In [293]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Choose and load data

In [622]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=0)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

address_runs = [x for x in all_subdir if 'DC01_RF08' in x]
print(address_runs)

> Répertoire : /home/adrian/RemoteFS/Rivendel/20201211/DC07_RF09/Try00
> Filename : Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 21, 30, 39, 49]
> slashcond | 5
No points deleted because they were not complete
Total number of data directories 2
[]


In [623]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC07_RF09']
000 > DC07_RF09
> condition names ['DC', 'RF']
> number of points 1
> N_ions = 1024
> e_GMol = 50


In [624]:
## Data loading (1D arrays)
# select data with init to reuse
forcetryinit = ['Try00']

data_name, num_runs \
= data_retrieve_RFRelax(all_subdir,points_and_coord,condition_parameters,slash_cfg,
                        mode='RFrelax') # forcecondnum = 0 , forcetryinit = forcetryinit

data_address = data_name[0]
data0 = data_name[1]
data6 = data_name[2]

> Points | 1
> Simulations pour chaque point | ['Try00', 'Try01']
['/home/adrian/RemoteFS/Rivendel/20201211/DC07_RF09/Try00', '/home/adrian/RemoteFS/Rivendel/20201211/DC07_RF09/Try01']
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/Rivendel/20201211/DC07_RF09/Try00\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG
00 - 01  >  /home/adrian/RemoteFS/Rivendel/20201211/DC07_RF09/Try01\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG
Time elapsed:  0.007169999999973697 s
Time elapsed:  0.00011949999999956162 m


# load context variables

In [625]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


In [626]:
## Data loading with hardcode names
eta = 1.5e-20

plot_point = 0
plot_try = 0
# str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
# str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
# str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
str_eta = '0.15D-19'
dirname_hc = str(data0[plot_point])+'/'
# Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
data0_hc    = '/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

['/home/adrian/RemoteFS/Rivendel/20201211/DC07_RF09/Try00\\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG', '/home/adrian/RemoteFS/Rivendel/20201211/DC07_RF09/Try01\\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG']/
Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5


In [627]:
# recover duration for each step of simu (i_free__fly, i_laser_fly ...)
plot_try = 1
address = all_subdir[plot_point]
print(address)
info_files = sort( [f for f in listdir(address) if isfile(join(address, f)) and ".info" in f] )
print(sort(info_files))


tmp_info = loadtxt(all_subdir[plot_point]+'/'+info_files[0],encoding ='utf-8',
            skiprows=1, usecols=(0),
            unpack=True)
eta = tmp_info[-2]
i_free__fly_Lan = tmp_info[-1]

tmp_info = loadtxt(all_subdir[0]+'/'+'xva_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.info',encoding ='utf-8',
            skiprows=0, usecols=(0),
            unpack=True)
i_laser_fly_init = tmp_info[-4]
i_laser_fly_quench = tmp_info[-3]
i_relax_fly = tmp_info[-1]

n_dt = 100
dt = 1/2e6

print('eta',eta)
print('i_free__fly_Lan',i_free__fly_Lan)
print('i_laser_fly_init',i_laser_fly_init)
print('i_laser_fly_quench',i_laser_fly_quench)
print('i_relax_fly',i_relax_fly)

/home/adrian/RemoteFS/Rivendel/20201211/DC07_RF09/Try00
['Langevin_cooling.info'
 'xva_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.info'
 'xva_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.info']
eta 1.5e-20
i_free__fly_Lan 1000.0
i_laser_fly_init 6000.0
i_laser_fly_quench 10000.0
i_relax_fly 75000.0


In [628]:
snap_num = int(i_relax_fly/real(i_snap_x+di_snap_x))
snap_time = []
for k in range(1,snap_num+1):
    snap_time.append(linspace(di_snap_x*k*dt,(di_snap_x*k+i_snap_x)*dt,i_snap_x)*1000)
print(shape(snap_time))

(24, 100)


# load and plot data

In [634]:
plot_point = 0
plot_try = 1
data_address_aux = data_address[plot_point][plot_try]
snap_files = [f for f in listdir(data_address_aux) if isfile(join(data_address_aux, f)) and "xN" in f]
str_load = data_address_aux + '/' + snap_files[0]
print(str_load)

fid = open(str_load, 'rb')

junk0 = fromfile(fid, int32, 1)  # Read record start tag
aux = fromfile(fid, int32, 1)
junk1 = fromfile(fid, int32, 1)  # Read record stop tag
i_snap_x = aux[0]

junk2 = fromfile(fid, int32, 1)  # Read record start tag
aux = fromfile(fid, int32, 1)
junk3 = fromfile(fid, int32, 1)  # Read record stop tag
di_snap_x = aux[0]

junk4 = fromfile(fid, int32, 1)  # Read record start tag
aux = fromfile(fid, int32, 1)
junk5 = fromfile(fid, int32, 1)  # Read record stop tag
jmax = aux[0]

junk6 = fromfile(fid, int32, 1)  # Read record start tag
aux = fromfile(fid, int32, 1)
junk7 = fromfile(fid, int32, 1)  # Read record stop tag
N_ions = aux[0]

junk8 = fromfile(fid, int32, 1)  # Read record start tag
r = fromfile(fid, float32, int(N_ions*3*snap_num*i_snap_x)) # jmax*N_ions*3 int(junk8[0]/4)
junk9 = fromfile(fid, int32, 1)  # Read record stop tag

junk10 = fromfile(fid, int32, 1)  # Read record start tag
xva = fromfile(fid, float32, int(snap_num*12*N_ions)) # jmax*N_ions*3
junk11 = fromfile(fid, int32, 1)  # Read record stop tag

junk12 = fromfile(fid, int32, 1)  # Read record start tag
t_act = fromfile(fid, float32, 24) # jmax*N_ions*3
junk13 = fromfile(fid, int32, 1)  # Read record stop tag

patate = fromfile(fid, int32, 1)  # Read record stop tag

print(i_snap_x,di_snap_x)
print(jmax,N_ions)
print(junk0[0],junk2[0],junk4[0],junk6[0],junk8[0],junk10[0],junk12[0],junk13[0])
print(patate)

/home/adrian/RemoteFS/Rivendel/20201211/DC07_RF09/Try01/xN_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.bin
100 3000
74257 1024
4 4 4 4 29491200 1179648 96 96
[]


In [635]:
t_act

array([0.01006513, 0.01163063, 0.01319612, 0.01476162, 0.01632712,
       0.01789263, 0.01945812, 0.02102363, 0.02258912, 0.02415463,
       0.02572013, 0.02728562, 0.02885113, 0.03041662, 0.03198212,
       0.03354762, 0.03511313, 0.03667862, 0.03824412, 0.03980963,
       0.04137513, 0.04294062, 0.04450613, 0.        ], dtype=float32)

In [469]:
# from scipy.io import FortranFile
# f = FortranFile( str_load, 'r' )
# f.read_ints( np.int32 )
# f.read_ints( np.int32 )
# f.read_ints( np.int32 )
# f.read_ints( np.int32 )
# auto_r = f.read_reals( dtype='float32' )
# auto_xva = f.read_reals( dtype='float32' )

array([0], dtype=int32)

In [549]:
r_reshape = reshape(r,(N_ions,3,snap_num*i_snap_x),order='F')
print(shape(r_reshape))
xva_reshape = reshape(xva,(snap_num,14,N_ions),order='F')
print(shape(xva_reshape))

(1024, 3, 2400)
(24, 14, 1024)


In [556]:
# Get T and fluo PM  All SimuType
plot_point = 0
plot_try = 3
address_reuseinit = all_subdir[plot_point]
address = data_address[plot_point][plot_try]
init_data = data0[plot_point][plot_try]
evol_data = data6[plot_point][plot_try]
print(init_data)
print(evol_data)

# onlyfiles = [f for f in listdir(evol_data) if isfile(join(evol_data, f)) and "SimuType" in f and ".dat" in f]
# print(sort(onlyfiles))
# onlyfiles_reuseinit = [f for f in listdir(init_data) if isfile(join(init_data, f)) and "SimuType" in f and ".dat" in f]
# print(sort(onlyfiles_reuseinit))

ta, te, ti = plot_T_and_PM_InitQ_Evol_AfterCool(address,
                                           flag_plot=0,fig_name='0o0o0o0',xlim1=(0,65),ylim1=(2e-4,6000),ylim2=(-50,46000))

fname = address[31:49]
plt.figure(re.sub('/','-',fname)).clf()
ax1 = subplot(111)
te_filter = []
for k in range(3):
    te_filter.append(savgol_filter(te[:,k], 25, 1))
    ax1.semilogy(ta*1e3,te_filter[k]) ; ax1.text(0,1e-4,'Filtered temperatures',fontsize = 10,ha='left')
    # ax1.semilogy(ta*1e3,te[:,k])
    ax1.grid()
    
ax1.set_xlabel('time [ms]')
ax1.set_ylabel('T [K]')
title(fname)
    
time_switch = i_free__fly_Lan *dt
ax1.vlines(time_switch*1e3,2e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly_init)*dt
ax1.vlines(time_switch*1e3,2e-4,1e4,color='green')
time_switch = (i_free__fly_Lan+i_laser_fly_init+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,2e-4,1e4,color='orange')
for k in range(snap_num):
    ax1.add_patch(
     patches.Rectangle(
        (snap_time[k][0]+time_switch*1e3, 1e-4),
        snap_time[k][-1]-snap_time[k][0],
        0.01,
        edgecolor = 'blue',
        facecolor = 'red',
        fill=True
     ) )
time_switch = (i_free__fly_Lan+i_laser_fly_init+i_laser_fly_quench+i_relax_fly)*dt
time_switch_old = (i_free__fly_Lan+i_laser_fly_init+i_laser_fly_quench)*dt
ax1.scatter(linspace(time_switch_old*1e3,(time_switch*1e3-(time_switch-time_switch_old)*1e3%1),int((time_switch-time_switch_old)*1e3)),[3e-2]*int((time_switch-time_switch_old)*1e3),marker = '+')
ax1.vlines(time_switch*1e3,2e-4,1e4,color='purple')

tight_layout()

# tt2, T_CM2, T_aux2, PM2
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

/home/adrian/RemoteFS/Rivendel/20201209/DC07_RF09/Try03\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20201209/DC07_RF09/Try03\Temp_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG
len save_T [55440]
[5.05000000e-07 2.02224108e-05 3.22467700e-05 8.56624579e-05
 4.72988007e+01 1.33382166e+00 6.66089307e+01]


In [557]:
time_switch_old = (i_free__fly_Lan+i_laser_fly_init+i_laser_fly_quench)*dt
num_to_1ms = (time_switch-time_switch_old)*1e3
print(num_to_1ms)
linspace(time_switch_old*1e3,time_switch*1e3,num_to_1ms)

37.5


/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  after removing the cwd from sys.path.


array([ 8.5       ,  9.54166667, 10.58333333, 11.625     , 12.66666667,
       13.70833333, 14.75      , 15.79166667, 16.83333333, 17.875     ,
       18.91666667, 19.95833333, 21.        , 22.04166667, 23.08333333,
       24.125     , 25.16666667, 26.20833333, 27.25      , 28.29166667,
       29.33333333, 30.375     , 31.41666667, 32.45833333, 33.5       ,
       34.54166667, 35.58333333, 36.625     , 37.66666667, 38.70833333,
       39.75      , 40.79166667, 41.83333333, 42.875     , 43.91666667,
       44.95833333, 46.        ])

In [563]:
print(shape(r_reshape))
print(i_snap_x)
print(i_snap_x*snap_num)

(1024, 3, 2400)
100
2400


In [621]:
plt.figure('my_little_cloud').clf()
ax1 = subplot(211)
ax2 = subplot(212)
time = 2299
for k in range(1024):
    ax1.scatter(r_reshape[k,0,time]*1000,r_reshape[k,1,time]*1000,color='xkcd:purplish blue',edgecolor='k')
    ax2.scatter(r_reshape[k,2,time]*1000,r_reshape[k,0,time]*1000,color='xkcd:purplish blue',edgecolor='k')
ax1.set_aspect("equal")

In [149]:
r_reshape[:,0,time]

array([ 1.9213607e-05,  1.7531773e-05,  9.0903268e-07, ...,
        6.2600761e-06,  4.8507595e-06, -2.8380939e-06], dtype=float32)

In [610]:
xva_reshape[:,12,25]-i_snap_x*dt*100

array([ 0.00506513,  0.00663063,  0.00819612,  0.00976162,  0.01132712,
        0.01289263,  0.01445812,  0.01602362,  0.01758913,  0.01915463,
        0.02072012,  0.02228563,  0.02385113,  0.02541662,  0.02698212,
        0.02854763,  0.03011313,  0.03167862,  0.03324413,  0.03480963,
        0.03637513,  0.03794063,  0.03950613, -0.005     ], dtype=float32)

In [603]:
snap_time[4]+(i_free__fly_Lan+i_laser_fly_init+i_laser_fly_quench)*dt*1e3

array([16.        , 16.00050505, 16.0010101 , 16.00151515, 16.0020202 ,
       16.00252525, 16.0030303 , 16.00353535, 16.0040404 , 16.00454545,
       16.00505051, 16.00555556, 16.00606061, 16.00656566, 16.00707071,
       16.00757576, 16.00808081, 16.00858586, 16.00909091, 16.00959596,
       16.01010101, 16.01060606, 16.01111111, 16.01161616, 16.01212121,
       16.01262626, 16.01313131, 16.01363636, 16.01414141, 16.01464646,
       16.01515152, 16.01565657, 16.01616162, 16.01666667, 16.01717172,
       16.01767677, 16.01818182, 16.01868687, 16.01919192, 16.01969697,
       16.02020202, 16.02070707, 16.02121212, 16.02171717, 16.02222222,
       16.02272727, 16.02323232, 16.02373737, 16.02424242, 16.02474747,
       16.02525253, 16.02575758, 16.02626263, 16.02676768, 16.02727273,
       16.02777778, 16.02828283, 16.02878788, 16.02929293, 16.02979798,
       16.03030303, 16.03080808, 16.03131313, 16.03181818, 16.03232323,
       16.03282828, 16.03333333, 16.03383838, 16.03434343, 16.03

In [596]:
asanyarray(snap_time)[:,-1]

array([ 1.55,  3.05,  4.55,  6.05,  7.55,  9.05, 10.55, 12.05, 13.55,
       15.05, 16.55, 18.05, 19.55, 21.05, 22.55, 24.05, 25.55, 27.05,
       28.55, 30.05, 31.55, 33.05, 34.55, 36.05])

In [174]:
time = 250
slice_min = -3e-5
slice_max = -slice_min
r_slice = []
for j,k in enumerate(r_reshape[:,0,time]):
    if k>slice_min and k<slice_max:
        r_slice.append(r_reshape[j,:,time])

plt.figure('my_little_cloud_slice_x').clf()
ax1 = subplot(111)
for k in range(shape(r_slice)[0]):
    ax1.scatter(r_slice[k][1]*1000,r_slice[k][2]*1000)
    
ax1.set_xlabel('y')
ax1.set_ylabel('z')

tight_layout()

In [173]:
time = 250
slice_min = -3e-5
slice_max = -slice_min
r_slice = []
for j,k in enumerate(r_reshape[:,2,time]):
    if k>slice_min and k<slice_max:
        r_slice.append(r_reshape[j,:,time])

plt.figure('my_little_cloud_slice_z').clf()
ax1 = subplot(111)
for k in range(shape(r_slice)[0]):
    ax1.scatter(r_slice[k][0]*1000,r_slice[k][1]*1000)
    
ax1.set_xlabel('x')
ax1.set_ylabel('y')
tight_layout()

In [286]:
# Load XYZ test
str_load = re.sub('Temp','xva',str(data6[plot_point][plot_try]))
str_load = '/home/adrian/RemoteFS/Rivendel/20201204/DC07_RF09/Try02/xva_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG'
print(str_load)
load_xyz_init_bin_DP(str_load)

/home/adrian/RemoteFS/Rivendel/20201204/DC07_RF09/Try02/xva_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG


(array([[-0.05581546, -0.04763122, -0.00763189, ...,  0.01884057,
         -0.00742793,  0.01564725],
        [-0.00288794,  0.03008364,  0.02287785, ..., -0.01687232,
          0.04041099, -0.01510458],
        [ 0.06114086,  0.00153408, -0.01091107, ...,  0.02227473,
          0.01879129,  0.03905647]]),
 array([[ 0.12035218, -0.44912634, -0.16657079, ..., -0.0685828 ,
         -0.43373072, -0.10810143],
        [-0.60057936, -0.34141979,  0.38419864, ..., -0.56866205,
         -0.26300209,  0.08432431],
        [-0.35625422,  0.29184962, -0.40708751, ..., -0.05072926,
         -0.01381154, -0.49651859]]),
 array([[ 2.26115274e+09,  1.93334509e+09,  3.08177361e+08, ...,
         -7.62201458e+08,  3.02631528e+08, -6.36800095e+08],
        [-1.60822441e+08,  1.64402969e+09,  1.24962173e+09, ...,
         -9.20768225e+08,  2.20185600e+09, -8.27742386e+08],
        [-1.24291811e+06,  7.55190649e+05,  8.03341983e+05, ...,
         -1.41597779e+06,  1.35377678e+06,  1.25060187e+06]]),
 arr

In [289]:
shape(load_xyz_init_bin_DP(str_load)[3])

(3, 1024)